In [ ]:
import os
import rdflib
import rdflib_jsonld
from rdflib import Literal
from rdflib import Graph, ConjunctiveGraph, plugin
from rdflib.serializer import Serializer

### Trying to parse an ISA json-ld instance file

In [ ]:
ontology = "wdt"

isa_json_ld_path = os.path.join("./output/BII-S-3-synth/", "isa-new_ids-BII-S-3-ld-" + ontology + "-v1.json")

with open(isa_json_ld_path) as LDin:
    print(LDin)    

In [ ]:
g3 = rdflib.Graph() # also tried ConjuctiveGraph (as per https://github.com/RDFLib/rdflib-jsonld/issues/53, no lucj)

In [ ]:

g3.parse(isa_json_ld_path, format='json-ld')

In [ ]:
# with open("BII-S-3.ttl", "w") as TTLout:

#     g3.serialize("BII-S-3.ttl", format='ttl')

### Getting the Data Files:

In [ ]:
# Select all the things (s) that are of type (rdf:type) c:
# qres = g3.query("""
# SELECT DISTINCT ?study ?protocol ?file ?filename WHERE { 
# ?study a wdt:Q101965 .
# ?protocol a wdt:Q41689629 . 
# ?file a wdt:Q5227290 .
# ?file wdtp:P527 ?filename .
# }""")

qres = g3.query("""
SELECT DISTINCT  ?file ?filename WHERE {  
?file a wdt:Q5227290 .
?file wdtp:has_name ?filename .
}""")

for row in qres:
    print(f"{row.file} | {row.filename}")

### Getting ISA Study Author Information:

In [ ]:
qres = g3.query("""
SELECT DISTINCT  ?person_iri ?lastname ?firstname ?email ?affiliation ?role WHERE {  
?person_iri a wdt:Q215627 .
?person_iri wdtp:P735 ?firstname .
?person_iri wdtp:P734 ?lastname .
?person_iri wdtp:P968 ?email .
?person_iri wdtp:P361 ?affiliation .
# ?person_iri wdtp:P2868 ?role_iri .
?role_iri wdtp:P527 ?role .
FILTER (str(?role) = "principal investigator role" ) .
}
""")

for row in qres:
    print(f"{row.firstname} {row.lastname} | {row.email} |  {row.affiliation} | {row.role}")

In [ ]:
qres = g3.query("""
SELECT DISTINCT  ?person_iri ?lastname ?firstname ?email ?affiliation ?role WHERE {  
?person_iri rdf:type wdt:Q215627 .
?person_iri wdtp:P735 ?firstname .
?person_iri wdtp:P734 ?lastname .
?person_iri wdtp:P968 ?email .
?person_iri wdtp:P361 ?affiliation .
?person_iri wdtp:P2868 ?role_iri .
?role_iri wdtp:P527 ?role .
#FILTER (regex(str(?role), “principal”) ) .
FILTER (str(?role) = "principal investigator role" ) .
}
""")

for row in qres:
    print(f"{row.firstname} {row.lastname} | {row.email} |  {row.affiliation} | {row.role}")

### Getting List of ISA Study Factors (Independent Variables)

In [ ]:
qres = g3.query("""
SELECT DISTINCT  ?factor_iri ?name ?type WHERE {  
?factor_iri a wdt:Q2487752 .
?factor_iri wdtp:P1448-FACTORNAME ?name .
#?factor_iri wdtp:P527 ?type .

}""")

for row in qres:
    print(f" {row.name} | {row.type}")

### Getting list of ISA Protocols and associated information

In [ ]:
qres = g3.query("""
SELECT DISTINCT  ?protocol_iri ?name ?protocol ?param_iri ?param_name ?onto_iri ?onto_src WHERE { 
?protocol_iri a wdt:Q41689629 .
?protocol_iri wdtp:has_name ?name .
?protocol_iri wdtp:P2283 ?param_iri .
?param_iri wdtp:P1448-PARAMNAME ?onto_iri .
?onto_iri wdtp:P527 ?param_name .
?onto_iri wdtp:P5967 ?onto_src .

}""")


for row in qres:
    print(f" | {row.protocol_iri} | {row.name} | {row.param_name} |  {row.onto_src} ")

### Getting list of ISA Study publications

In [ ]:
qres = g3.query("""
SELECT DISTINCT  ?study_iri ?study_title ?publication_iri ?pmid ?doi ?name ?authorlist ?title WHERE {
?study_iri a wdt:Q101965 .
?study_iri wdtp:Q853614 ?study_id .
?study_iri wdtp:P1476 ?study_title .
?study_iri wdtp:P527 ?publication_iri .
?publication_iri wdtp:P1476 ?title .
?publication_iri wdtp:P527 ?doi .
?publication_iri wdtp:P527 ?pmid .
?publication_iri wdtp:P527 ?authorlist .
}""")


for row in qres:
    print(f"  {row.study_title}  |  {row.publication_iri} |  {row.title}  | {row.pmid} | {row.doi} |  {row.authorlist}")

### Getting list of ISA Source material properties

In [ ]:
# qres = g3.query("""
# SELECT DISTINCT  ?study_iri  WHERE {
# ?study_iri a wdt:Q101965 .
# ?study_iri wdtp:Q853614 ?study_id .
# ?study_iri wdtp:P1476 ?study_title .
# ?study_iri wdtp:P527 ?publication_iri .
# }""")


# for row in qres:
#     print(f"  {row.title}  | {row.pmid} | {row.doi} |  {row.authorlist}")

### Getting a simple JSON-LD file

In [ ]:
uri ='https://raw.githubusercontent.com/json-ld/json-ld.org/main/examples/syntax/example-058-Implicitly-named-graph.json'

### Creating a graph object

In [ ]:
graph = rdflib.Graph()

#### Parsing a jsonld document and loading the document as a graph in rdflib

In [ ]:
graph.parse(uri, format='json-ld')

#### Converting the graph to the Turtle format

In [ ]:
new_data_as_turtle = graph.serialize(format='ttl').decode('u8)')

In [ ]:
print(new_data_as_turtle)

In [ ]:
g1 = rdflib.Graph()
g1.parse('https://raw.githubusercontent.com/json-ld/json-ld.org/main/examples/syntax/example-058-Implicitly-named-graph.json', format='json-ld')

In [ ]:
g1_as_turtle = g1.serialize(format='ttl').decode('u8)')

In [ ]:
print(g1_as_turtle)

### Using `ConjuctiveGraph` instead of `Graph` for parsing an ontology file with import statements

In [ ]:
g2 = rdflib.ConjunctiveGraph()
g2.parse("http://purl.obolibrary.org/obo/iao.owl")
j = g2.serialize(format='json-ld', indent=4).decode('u8')
with open('iao-ontology.jsonld', 'w') as f:
    f.write(str(j))
f.close()

In [ ]:
print(g2.serialize(format='json-ld', indent=4).decode('u8'))